### First let's import our libraries and put our CSVs into Pandas Dataframes. You can run this Notebook by updating the location of the CSVs.

In [650]:
import pandas as pd
import plotly.plotly as py
import plotly.offline as pyo
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [651]:
customers_df = pd.read_csv('/Users/utkarshdalal/workspace/worldcover-coding-challenge/tables/wc_customers.csv')
policies_df = pd.read_csv('/Users/utkarshdalal/workspace/worldcover-coding-challenge/tables/wc_customer_policies.csv')
communities_df = pd.read_csv('/Users/utkarshdalal/workspace/worldcover-coding-challenge/tables/wc_communities.csv')
community_payouts_df = pd.read_csv('/Users/utkarshdalal/workspace/worldcover-coding-challenge/tables/wc_community_payouts.csv')
policy_transactions_df = pd.read_csv('/Users/utkarshdalal/workspace/worldcover-coding-challenge/tables/wc_policy_transactions.csv')

### As a side note, any legend entry in the plots below can be clicked to enable/disable its values in the plot 

## 1) Our first task is to see if retention patterns vary with geography.

### Let's explore the policy data, specifically how many were purchased by season

In [652]:
data = [go.Histogram(x=policies_df['season'])]
layout = go.Layout(
    title='Num Policies by Season',
    yaxis=dict(
        title='Num Policies'
    )
)
pyo.iplot(go.Figure(data=data, layout=layout))

### The majority of the policies seem to have been purchased for the 2018 Major season. There's a huge drop in purchases from 2018 to 2019. Maybe this is because farmers purchase them close to the season and our data is not up to date? Let's see.

In [596]:
policies_by_season_and_date = policies_df.groupby(['season', 'date_issued'])['customer_policy_id'].count()

trace1 = go.Bar(x=policies_by_season_and_date['2016 major'].index, y=policies_by_season_and_date['2016 major'].values, name="2016 Major Issue Date")
trace2 = go.Bar(x=policies_by_season_and_date['2017 major'].index, y=policies_by_season_and_date['2017 major'].values, name="2017 Major Issue Date")
trace3 = go.Bar(x=policies_by_season_and_date['2018 major'].index, y=policies_by_season_and_date['2018 major'].values, name="2018 Major Issue Date")
trace4 = go.Bar(x=policies_by_season_and_date['2018 minor'].index, y=policies_by_season_and_date['2018 minor'].values, name="2018 Minor Issue Date")
trace5 = go.Bar(x=policies_by_season_and_date['2019 major'].index, y=policies_by_season_and_date['2019 major'].values, name="2019 Major Issue Date")
trace6 = go.Bar(x=policies_by_season_and_date['2019 minor'].index, y=policies_by_season_and_date['2019 minor'].values, name="2019 Minor Issue Date")

data = [trace1, trace2, trace3, trace4, trace5, trace6]

layout = go.Layout(
    title='Num Policies by Issue Date',
    yaxis=dict(
        title='Num Policies Issued'
    ),
    xaxis=dict(
        title='Date'
    )
)
pyo.iplot(go.Figure(data=data, layout=layout))

### In 2018, most policies were purchased before June, and we have data up to July 2019 - so it looks like the drop in policy purchases is genuine. A quick Google search tells us there was a drought in Ghana in the early months of 2018, possibly leading to the spike. 

### Let's visualise these purchases geographically. It should be noted that 500 or so communities do not have a lat/lon, so these will not be plotted below.

In [662]:
buyers_by_season_df = pd.merge(policies_df, customers_df, on='customer_id').groupby(['season', 'community_id'])['customer_id'].count().reset_index()
buyers_by_season_df = pd.merge(buyers_by_season_df, communities_df, on='community_id')
buyers_by_season_df['plot_text'] = buyers_by_season_df['community_name'] + '<br>Count: ' + buyers_by_season_df['customer_id'].astype(str)

In [670]:
# Due to an issue with plotly, we have to add a large point off-map for each season for the legend to show correctly

df_2016 = buyers_by_season_df[buyers_by_season_df['season'] == '2016 major']
df_2017 = buyers_by_season_df[buyers_by_season_df['season'] == '2017 major']
df_2018 = buyers_by_season_df[buyers_by_season_df['season'] == '2018 major']
df_2019 = buyers_by_season_df[buyers_by_season_df['season'] == '2019 major']
df_2018_minor = buyers_by_season_df[buyers_by_season_df['season'] == '2018 minor']
df_2019_minor = buyers_by_season_df[buyers_by_season_df['season'] == '2019 minor']
scale = 4.0

data_2016 = [ dict(
    lat = df_2016['latitude'].append(pd.Series([37.0])),
    lon = df_2016['longitude'].append(pd.Series([-122.0])),
    text = df_2016['plot_text'].astype(str),
    name = '2016 Major Season Customers',
    marker = dict(
        size = (df_2016['customer_id']/scale).append(pd.Series([500.0]))
    ),
    type = 'scattergeo'
) ]

data_2017 = [ dict(
    lat = df_2017['latitude'].append(pd.Series([37.0])),
    lon = df_2017['longitude'].append(pd.Series([-122.0])),
    text = df_2017['plot_text'].astype(str),
    name = '2017 Major Season Customers',
    marker = dict(
        size = (df_2017['customer_id']/scale).append(pd.Series([700.0]))
    ),
    type = 'scattergeo'
) ]

data_2018 = [ dict(
    lat = df_2018['latitude'].append(pd.Series([37.0])),
    lon = df_2018['longitude'].append(pd.Series([-122.0])),
    text = df_2018['plot_text'].astype(str),
    name = '2018 Major Season Customers',
    marker = dict(
        size = (df_2018['customer_id']/scale).append(pd.Series([1000.0]))
    ),
    type = 'scattergeo'
) ]

data_2019 = [ dict(
    lat = df_2019['latitude'].append(pd.Series([37.0])),
    lon = df_2019['longitude'].append(pd.Series([-122.0])),
    text = df_2019['plot_text'].astype(str),
    name = '2019 Major Season Customers',
    marker = dict(
        size = (df_2019['customer_id']/scale).append(pd.Series([700.0]))
    ),
    type = 'scattergeo'
) ]

data_2018_minor = [ dict(
    lat = df_2018_minor['latitude'].append(pd.Series([37.0])),
    lon = df_2018_minor['longitude'].append(pd.Series([-122.0])),
    text = df_2018_minor['plot_text'].astype(str),
    name = '2018 Minor Season Customers',
    marker = dict(
        size = (df_2018_minor['customer_id']/scale).append(pd.Series([700.0]))
    ),
    type = 'scattergeo'
) ]

data_2019_minor = [ dict(
    lat = df_2019_minor['latitude'].append(pd.Series([37.0])),
    lon = df_2019_minor['longitude'].append(pd.Series([-122.0])),
    text = df_2019_minor['plot_text'].astype(str),
    name = '2019 Minor Season Customers',
    marker = dict(
        size = (df_2019_minor['customer_id']/scale).append(pd.Series([200.0]))
    ),
    type = 'scattergeo'
) ]
    
layout = go.Layout(
    title = 'Buyers Across Seasons',
    geo = dict(
        resolution = 110,
        scope = 'africa',
        showframe = False,
        showcoastlines = True,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        center = {'lat': 9.139, 'lon': -1.143},
        projection = go.layout.geo.Projection(
            type = 'mercator',
            scale=20
        )
    )
)

fig = go.Figure(layout=layout, data=data_2016+data_2017+data_2018+data_2019+data_2018_minor+data_2019_minor)
url = pyo.iplot(fig)

### Interestingly, with the exception of the 2018 Major season, policy purchases for each season appear to be concentrated in specific areas

### Next, let's try and separate retained buyers. Here we define a retained buyer as a farmer that has purchased a policy for more than one season. 

In [101]:
times_policy_purchased_df = policies_df.groupby('customer_id')['season'].count()
single_buyer_df = times_policy_purchased_df[times_policy_purchased_df == 1]
repeat_buyer_df = times_policy_purchased_df[times_policy_purchased_df > 1]

### We then find the number of retained and one-time buyers by community so we can visualise them.

In [208]:
single_buyer_count_by_community = pd.merge(single_buyer_df, customers_df, on='customer_id').groupby('community_id').count()
single_buyer_count_by_community = pd.merge(single_buyer_count_by_community, communities_df, on='community_id')
single_buyer_count_by_community['plot_text'] = single_buyer_count_by_community['community_name'] + '<br>Count: ' + single_buyer_count_by_community['customer_id'].astype(str)

In [167]:
repeat_buyer_count_by_community = pd.merge(repeat_buyer_df, customers_df, on='customer_id').groupby('community_id').count()
repeat_buyer_count_by_community = pd.merge(repeat_buyer_count_by_community, communities_df, on='community_id')
repeat_buyer_count_by_community['plot_text'] = repeat_buyer_count_by_community['community_name'] + '<br>Count: ' + repeat_buyer_count_by_community['customer_id'].astype(str)

In [233]:
buyers_by_season_df = pd.merge(policies_df, customers_df, on='customer_id').groupby(['season', 'community_id'])['customer_id'].count().reset_index()
buyers_by_season_df = pd.merge(buyers_by_season_df, communities_df, on='community_id')
buyers_by_season_df['plot_text'] = buyers_by_season_df['community_name'] + '<br>Count: ' + repeat_buyer_count_by_community['customer_id'].astype(str)

In [679]:
single_buyer_data = [ dict(
    lat = single_buyer_count_by_community['latitude'].append(pd.Series([37.0])),
    lon = single_buyer_count_by_community['longitude'].append(pd.Series([-122.0])),
    text = single_buyer_count_by_community['plot_text'].astype(str),
    name = 'Number of One-time Customers',
    marker = dict(
        color = 'red',
        size = (single_buyer_count_by_community['customer_id']/3.0).append(pd.Series([200.0]))
    ),
    type = 'scattergeo'
) ]

repeat_buyer_data = [ dict(
    lat = repeat_buyer_count_by_community['latitude'].append(pd.Series([37.0])),
    lon = repeat_buyer_count_by_community['longitude'].append(pd.Series([-122.0])),
    text = repeat_buyer_count_by_community['plot_text'].astype(str),
    name = 'Number of Retained Customers',
    marker = dict(
        color = 'blue',
        size = (repeat_buyer_count_by_community['customer_id']/3.0).append(pd.Series([1000.0]))
    ),
    type = 'scattergeo'
) ]
    
layout = go.Layout(
    title = 'Retained vs One-time Customers by Community',
    geo = dict(
        resolution = 110,
        scope = 'africa',
        showframe = False,
        showcoastlines = True,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        center = {'lat': 9.139, 'lon': -1.143},
        projection = go.layout.geo.Projection(
            type = 'mercator',
            scale=20
        ),
    )
)

fig = go.Figure(layout=layout, data=single_buyer_data+repeat_buyer_data)
url = pyo.iplot(fig)

### We see there are some hotspots with a large number of retained buyers, but what's important is the retention rate - not the absolute number. Next we plot the retention rate geographically by community.

In [284]:
buyers_by_community = pd.merge(single_buyer_count_by_community, repeat_buyer_count_by_community, on='community_id', suffixes=('_single', '_repeat'))

In [294]:
buyers_by_community['total_buyers'] = buyers_by_community['customer_id_single'] + buyers_by_community['customer_id_repeat']
buyers_by_community['retention_rate'] = buyers_by_community['customer_id_repeat']/buyers_by_community['total_buyers'] * 100.0
buyers_by_community['plot_text'] = buyers_by_community['community_name_single'] + '<br>Retention Rate: ' + buyers_by_community['retention_rate'].astype(str) + '%'

In [601]:
mean_retention_rate = buyers_by_community['retention_rate'].mean()
mean_retention_rate
# Note that this is the mean retention rate by community, not by farmer

29.655400194126223

In [636]:
retention_rate_data = [ dict(
    lat = buyers_by_community['latitude_single'],
    lon = buyers_by_community['longitude_single'],
    text = buyers_by_community['plot_text'].astype(str),
    name = 'Retention Rate',
    marker = dict(
        color = 'red',
        size = buyers_by_community['retention_rate']/5.0
    ),
    type = 'scattergeo'
) ]
    
layout = go.Layout(
    title = 'Retention Rate Geographically by Community',
    geo = dict(
        resolution = 110,
        scope = 'africa',
        showframe = False,
        showcoastlines = True,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        center = {'lat': 9.139, 'lon': -1.143},
        projection = go.layout.geo.Projection(
            type = 'mercator',
            scale=20
        ),
    )
)

fig = go.Figure(layout=layout, data=retention_rate_data)
url = pyo.iplot(fig)

### This is helpful, but it's still difficult to separate which communities are more likely to have retained farmers. Let's plot communities with higher than average and lower than average retention rates.

In [635]:
below_avg_retention_buyers = buyers_by_community[buyers_by_community['retention_rate'] <= mean_retention_rate]
below_avg_retention_rate_data = [ dict(
    lat = below_avg_retention_buyers['latitude_single'],
    lon = below_avg_retention_buyers['longitude_single'],
    text = below_avg_retention_buyers['plot_text'].astype(str),
    name = 'Retention Rate Below Average',
    marker = dict(
        color = 'red',
        size = 5
    ),
    type = 'scattergeo'
) ]

above_avg_retention_buyers = buyers_by_community[buyers_by_community['retention_rate'] > mean_retention_rate]
above_avg_retention_rate_data = [ dict(
    lat = above_avg_retention_buyers['latitude_single'],
    lon = above_avg_retention_buyers['longitude_single'],
    text = above_avg_retention_buyers['plot_text'].astype(str),
    name = 'Retention Rate Above Average',
    marker = dict(
        color = 'blue',
        size = 5
    ),
    type = 'scattergeo'
) ]
    
layout = go.Layout(
    title = 'Communities with Above/Below Average Retention Rates',
    geo = dict(
        resolution = 110,
        scope = 'africa',
        showframe = False,
        showcoastlines = True,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        center = {'lat': 9.139, 'lon': -1.143},
        projection = go.layout.geo.Projection(
            type = 'mercator',
            scale=20
        ),
    )
)

fig = go.Figure(layout=layout, data=below_avg_retention_rate_data+above_avg_retention_rate_data)
url = pyo.iplot(fig)

### Above we see communities with higher than average retention rates in blue, and lower than average retention rates in red. 
## There does not seem to be any geographical pattern to retention rates.

## 2) Our next task is identifying socio-demographic factors that influence retention

### For this, we find the overall retention rate (different from the mean retention rate by community we found earlier) and compare it to the retention rate for sub-sections of the population.

In [605]:
# Socio-economic factors influencing retention
x = customers_df
x['retained_customer'] = x['customer_id'].isin(repeat_buyer_df.index)
total_customers = x['customer_id'].count()
total_retention_rate = x[x['retained_customer'] == True]['customer_id'].count()/total_customers * 100.0
total_retention_rate

15.689056308399785

### First we look at retention rates by size of farm. Almost all the farms (20k) are 13 acres in area, which is why 13 acre farms are a distinct category. This also means that this may not be a useful comparison, since non-13 acre farms are few in number.

In [681]:
customers_df.groupby('farm_size').count()

,customer_id,community_id,gender,literacy,has_mobile_money,registration_date,created_at,updated_at,ussd_created,has_phone
farm_size,,,,,,,,,,
1.0,12,12,12,12,12,12,12,3,12,12
2.0,63,63,63,63,54,63,63,8,63,63
2.5,1,1,1,1,0,1,1,0,1,1
3.0,36,36,36,36,29,36,36,11,36,36
4.0,58,58,58,58,19,58,58,18,58,58
5.0,36,36,36,36,20,36,36,17,36,36
6.0,44,44,44,44,17,44,44,9,44,44
7.0,25,25,25,25,8,25,25,12,25,25
8.0,13,13,13,13,3,13,13,4,13,13


In [606]:
farms_less_than_10 = x[(x['farm_size'] < 10.0)]
farms_less_than_10_customers = farms_less_than_10['customer_id'].count()
farms_less_than_10_retention_rate = farms_less_than_10[farms_less_than_10['retained_customer']]['customer_id'].count()/farms_less_than_10_customers * 100.0
farms_less_than_10_retention_rate

16.387959866220736

In [607]:
farms_13_to_20 = x[(x['farm_size'] > 13.0) & (x['farm_size'] < 20.0)]
farms_13_to_20_customers = farms_13_to_20['customer_id'].count()
farms_13_to_20_retention_rate = farms_13_to_20[farms_13_to_20['retained_customer']]['customer_id'].count()/farms_13_to_20_customers * 100.0

In [608]:
farms_13 = x[(x['farm_size'] == 13.0)]
farms_13_customers = farms_13['customer_id'].count()
farms_13_retention_rate = farms_13[farms_13['retained_customer']]['customer_id'].count()/farms_13_customers * 100.0

In [609]:
farms_10_to_13 = x[(x['farm_size'] >= 10.0) & (x['farm_size'] < 13.0)]
farms_10_to_13_customers = farms_10_to_13['customer_id'].count()
farms_10_to_13_retention_rate = farms_10_to_13[farms_10_to_13['retained_customer']]['customer_id'].count()/farms_10_to_13_customers * 100.0

In [610]:
farms_20_to_30 = x[(x['farm_size'] >= 20.0) & (x['farm_size'] < 30.0)]
farms_20_to_30_customers = farms_20_to_30['customer_id'].count()
farms_20_to_30_retention_rate = farms_20_to_30[farms_20_to_30['retained_customer']]['customer_id'].count()/farms_20_to_30_customers * 100.0

In [611]:
farms_greater_than_30 = x[(x['farm_size'] >= 30.0)]
farms_greater_than_30_customers = farms_greater_than_30['customer_id'].count()
farms_greater_than_30_retention_rate = farms_greater_than_30[farms_greater_than_30['retained_customer']]['customer_id'].count()/farms_greater_than_30_customers * 100.0

In [640]:
retention_rates = ['All Farms', 'Farms < 10 Acres', 'Farms 10-13 Acres', 'Farms 13 Acres', 'Farms 13-20 Acres', 'Farms 20-30 Acres', 'Farms > 30 Acres']

trace1 = go.Bar(x=retention_rates, y=[total_retention_rate, farms_less_than_10_retention_rate, farms_10_to_13_retention_rate, farms_13_retention_rate, farms_13_to_20_retention_rate, farms_20_to_30_retention_rate, farms_greater_than_30_retention_rate], name="Retention Rates")
trace2 = go.Bar(x=retention_rates, y=[total_customers, farms_less_than_10_customers, farms_10_to_13_customers, farms_13_customers, farms_13_to_20_customers, farms_20_to_30_customers, farms_greater_than_30_customers], name="Num Customers")

data = [trace1]
layout = go.Layout(
    title='Retention Rates by Farm Size',
    xaxis=dict(
        title='Farm Size'
    ),
    yaxis=dict(
        title='% of Customers Retained'
    )
)
pyo.iplot(go.Figure(data=data, layout=layout))

### We see that very large and very small farms, as well as farms between 13 and 20 acres are more likely to be owned by retained customers, however once again the number of farms which are not 13 acres in size are so small that this is most likely innacurate.

### Next we look at retention rates by literacy.

In [433]:
customers_df.groupby('literacy').count()

,customer_id,community_id,gender,has_mobile_money,registration_date,farm_size,created_at,updated_at,ussd_created,has_phone,retained_customer
literacy,,,,,,,,,,,
national_r,11,11,11,5,11,11,11,8,11,11,11
national_rw,12,12,12,10,12,12,12,12,12,12,12
native_r,265,265,265,230,265,265,265,248,265,265,265
native_rw,15473,15473,2860,2097,15470,15473,15473,6865,15473,15473,15473
none,4680,4680,4680,1886,4680,4680,4680,3165,4680,4680,4680


In [614]:
farms_national_r_literacy = x[(x['literacy'] == 'national_r')]
farms_national_r_literacy_customers = farms_national_r_literacy['customer_id'].count()
farms_national_r_literacy_retention_rate = farms_national_r_literacy[farms_national_r_literacy['retained_customer']]['customer_id'].count()/farms_national_r_literacy_customers * 100.0
farms_national_r_literacy_retention_rate

36.36363636363637

In [615]:
farms_national_rw_literacy = x[(x['literacy'] == 'national_rw')]
farms_national_rw_literacy_customers = farms_national_rw_literacy['customer_id'].count()
farms_national_rw_literacy_retention_rate = farms_national_rw_literacy[farms_national_rw_literacy['retained_customer']]['customer_id'].count()/farms_national_rw_literacy_customers * 100.0
farms_national_rw_literacy_retention_rate

33.33333333333333

In [616]:
farms_native_r_literacy = x[(x['literacy'] == 'native_r')]
farms_native_r_literacy_customers = farms_native_r_literacy['customer_id'].count()
farms_native_r_literacy_retention_rate = farms_native_r_literacy[farms_native_r_literacy['retained_customer']]['customer_id'].count()/farms_native_r_literacy_customers * 100.0
farms_native_r_literacy_retention_rate

23.77358490566038

In [617]:
farms_native_rw_literacy = x[(x['literacy'] == 'native_rw')]
farms_native_rw_literacy_customers = farms_native_rw_literacy['customer_id'].count()
farms_native_rw_literacy_retention_rate = farms_native_rw_literacy[farms_native_rw_literacy['retained_customer']]['customer_id'].count()/farms_native_rw_literacy_customers * 100.0
farms_native_rw_literacy_retention_rate

13.158404963484779

In [618]:
farms_none_literacy = x[(x['literacy'] == 'none')]
farms_none_literacy_customers = farms_none_literacy['customer_id'].count()
farms_none_literacy_retention_rate = farms_none_literacy[farms_none_literacy['retained_customer']]['customer_id'].count()/farms_none_literacy_customers * 100.0
farms_none_literacy_retention_rate

23.504273504273502

In [626]:
retention_rates = ['All Customers', 'Natl Language Read', 'Natl Language Write', 'Native Language Read', 'Native Language Write', 'Illiterate Customers']

trace1 = go.Bar(x=retention_rates, y=[total_retention_rate, farms_national_r_literacy_retention_rate, farms_national_rw_literacy_retention_rate, farms_native_r_literacy_retention_rate, farms_native_rw_literacy_retention_rate, farms_none_literacy_retention_rate], name="Retention Rates")

data = [trace1]

layout = go.Layout(
    title='Retention Rates by Literacy',
    xaxis=dict(
        title='Farmer Literacy'
    ),
    yaxis=dict(
        title='% of Customers Retained'
    )
)
pyo.iplot(go.Figure(data=data, layout=layout))
# Customers who are not R+W native language are more likely to be retained. However, most can R+W native language so this skews data

### Interestingly, farmers with no literacy are more likely than average to be retained. Farmers who can read/write their national language or can read their native language are also more likely to be retained; however, again these farmers are few in number so this may not be accurate.

### Phone ownership is next

In [682]:
customers_df.groupby('has_phone').count()

,customer_id,community_id,gender,literacy,has_mobile_money,registration_date,farm_size,created_at,updated_at,ussd_created
has_phone,,,,,,,,,,
False,16582,16582,5326,16582,1423,16581,16582,16582,6896,16582
True,3859,3859,2502,3859,2805,3857,3859,3859,3402,3859


In [627]:
customes_with_phones = x[(x['has_phone'] == True)]
customes_with_phones_customers = customes_with_phones['customer_id'].count()
customes_with_phones_retention_rate = customes_with_phones[customes_with_phones['retained_customer']]['customer_id'].count()/customes_with_phones_customers * 100.0
customes_with_phones_retention_rate

19.383259911894275

In [628]:
customes_without_phones = x[(x['has_phone'] == False)]
customes_without_phones_customers = customes_without_phones['customer_id'].count()
customes_without_phones_retention_rate = customes_without_phones[customes_without_phones['retained_customer']]['customer_id'].count()/customes_without_phones_customers * 100.0
customes_without_phones_retention_rate

14.829333011699433

In [639]:
retention_rates = ['All Customers', 'Customers with Phones', 'Customers without Phones']

trace1 = go.Bar(x=retention_rates, y=[total_retention_rate, customes_with_phones_retention_rate, customes_without_phones_retention_rate], name="Retention Rates")

data = [trace1]

layout = go.Layout(
    title='Retention Rates by Phone Ownership',
    yaxis=dict(
        title='% of Customers Retained'
    )
)

pyo.iplot(go.Figure(data=data, layout=layout))

# Customers with phones are more likely to be retained

### Farmers with phones are more likely to be retained.

### What about gender?

In [683]:
customers_df.groupby('gender').count()

,customer_id,community_id,literacy,has_mobile_money,registration_date,farm_size,created_at,updated_at,ussd_created,has_phone
gender,,,,,,,,,,
F,2904,2904,2904,702,2904,2904,2904,2050,2904,2904
M,4924,4924,4924,2143,4924,4924,4924,3692,4924,4924


In [631]:
female = x[(x['gender'] == 'F')]
female_customers = female['customer_id'].count()
female_retention_rate = female[female['retained_customer']]['customer_id'].count()/female_customers * 100.0
female_retention_rate

19.696969696969695

In [632]:
male = x[(x['gender'] == 'M')]
male_customers = male['customer_id'].count()
male_retention_rate = male[male['retained_customer']]['customer_id'].count()/male_customers * 100.0
male_retention_rate

21.039805036555645

In [633]:
no_gender = x[(x['gender'].isna())]
no_gender_customers = no_gender['customer_id'].count()
no_gender_retention_rate = no_gender[no_gender['retained_customer']]['customer_id'].count()/no_gender_customers * 100.0
no_gender_retention_rate

12.677396337112503

In [638]:
retention_rates = ['All Customers', 'Female Customers', 'Male Customers', 'Unknown Gender Customers']

trace1 = go.Bar(x=retention_rates, y=[total_retention_rate, male_retention_rate, female_retention_rate, no_gender_retention_rate], name="Retention Rates")

data = [trace1]

layout = go.Layout(
    title='Retention Rates by Gender',
    yaxis=dict(
        title='% of Customers Retained'
    )
)

pyo.iplot(go.Figure(data=data, layout=layout))

# Customers with a known gender are more likely to be retained. How is this data collected?

### Here we see that any farmer with an assigned gender is more likely than average to be retained. Is there something in the way this data is collected that is causing this? Is it self-reported?

### Finally, we examine the effect that having mobile money has

In [684]:
customers_df.groupby('has_mobile_money').count()

,customer_id,community_id,gender,literacy,registration_date,farm_size,created_at,updated_at,ussd_created,has_phone
has_mobile_money,,,,,,,,,,
False,3305,3305,2116,3305,3305,3305,3305,2383,3305,3305
True,923,923,729,923,921,923,923,665,923,923


In [642]:
mobile_money = x[(x['has_mobile_money'] == True)]
mobile_money_customers = mobile_money['customer_id'].count()
mobile_money_retention_rate = mobile_money[mobile_money['retained_customer']]['customer_id'].count()/mobile_money_customers * 100.0
mobile_money_retention_rate

30.87757313109426

In [643]:
no_mobile_money = x[(x['has_mobile_money'] == False)]
no_mobile_money_customers = no_mobile_money['customer_id'].count()
no_mobile_money_retention_rate = no_mobile_money[no_mobile_money['retained_customer']]['customer_id'].count()/no_mobile_money_customers * 100.0
no_mobile_money_retention_rate

20.090771558245084

In [644]:
unknown_mobile_money = x[(x['has_mobile_money'].isna())]
unknown_mobile_money_customers = unknown_mobile_money['customer_id'].count()
unknown_mobile_money_retention_rate = unknown_mobile_money[unknown_mobile_money['retained_customer']]['customer_id'].count()/unknown_mobile_money_customers * 100.0
unknown_mobile_money_retention_rate

13.927095540615555

In [646]:
retention_rates = ['All Customers', 'Customers w Mobile Money', 'Customers w/out Mobile Money', 'Unknown Mobile Money Customers']

trace1 = go.Bar(x=retention_rates, y=[total_retention_rate, mobile_money_retention_rate, no_mobile_money_retention_rate, unknown_mobile_money_retention_rate], name="Retention Rates")

data = [trace1]

layout = go.Layout(
    title='Retention Rates by Mobile Money Possession',
    yaxis=dict(
        title='% of Customers Retained'
    )
)
pyo.iplot(go.Figure(data=data, layout=layout))

# Customers with a known mobile money are more likely to be retained. How is this data collected?

### Like with gender, we see that farmers for which we have data for mobile money possession are more likely to be retained. Again, perhaps there is something in the way this data is collected that is behind this.

## 3) Lastly, we examine the effects of experiencing droughts and receiving payouts on retention and premiums

### Here, we only consider whether policies were paid out - we disregard policies where payouts are requested or due.

In [492]:
policies_df.groupby('status').count()

,customer_policy_id,date_issued,created_at,updated_at,customer_id,date_planted,date_priced,crop,season,retained_customer
status,,,,,,,,,,
active,210,210,210,210,210,161,210,210,210,210
dispute,49,49,49,49,49,37,0,49,49,49
expired,19780,19780,19780,19780,19780,6793,14192,19780,19780,19780
paid out,1579,1579,1579,1579,1579,822,1574,1579,1579,1579
payout due,678,678,678,678,678,596,678,678,678,678
payout requested,58,58,58,58,58,57,58,58,58,58
pending,1632,1632,1632,247,1632,52,6,1632,1632,1632
planted,156,156,156,154,156,155,0,156,156,156
priced,2,2,2,2,2,2,2,2,2,2


### To do this, we find the first time that a customer was paid out, and then we see if that customer purchased a policy in a future season. 

In [498]:
seasons_to_num= {'2016 major': 1, '2017 major': 2, '2018 major': 3, '2018 minor': 4, '2019 major': 5, '2019 minor': 6}

policies_with_payouts_df = policies_df.copy()
policies_with_payouts_df['paid_out'] = policies_with_payouts_df['status'] == 'paid out'
policies_with_payouts_df['season_num'] = policies_with_payouts_df['season'].map(lambda x: seasons_to_num[x])

In [547]:
sorted_policies_with_payouts_df = policies_with_payouts_df.sort_values(by='season')
first_payout_by_customer = sorted_policies_with_payouts_df[sorted_policies_with_payouts_df['paid_out']].groupby(['customer_id', 'season']).first().reset_index()
paid_out_customers = first_payout_by_customer[['customer_id', 'season', 'season_num']]
num_paid_out_customers = paid_out_customers['customer_id'].count()
# This is the number of customers who were paid out

In [548]:
num_paid_out_customers

1569

In [534]:
paid_out_customers.groupby('season').count()
# Almost all payouts happened in the 2018 major season

,customer_id,season_num
season,,
2016 major,3,3
2018 major,1538,1538
2018 minor,26,26
2019 major,2,2


### However, we see that most payouts happened in the 2018 major season. Since not many seasons have passed after that, we may not have enough data to say definitively what effect this has on retention rates.
### At first glance though, receiving a payout appears to have a strong negative effect on retention rates. Only 65 of 1569 customers who received a payout took out a policy for a future season.

In [685]:
joined_payout_df = policies_with_payouts_df.join(paid_out_customers, on=['customer_id'], lsuffix='', rsuffix='_paid_out', how='inner')
policies_after_payout = joined_payout_df[joined_payout_df['season_num'] > joined_payout_df['season_num_paid_out']]
num_customers_after_payout = len(policies_after_payout['customer_id'].unique())
num_customers_after_payout

65

In [649]:
retention_rates = ['All Customers', 'Customers with Prior Payout']

trace1 = go.Bar(x=retention_rates, y=[total_retention_rate, num_customers_after_payout/num_paid_out_customers * 100.0], name="Retention Rates")

data = [trace1]

layout = go.Layout(
    title='Retention Rates by Past Payout',
    yaxis=dict(
        title='% of Customers Retained'
    )
)

pyo.iplot(go.Figure(data=data, layout=layout))

# Customers with a prior payout are much less likely than average to be retained. However, most payouts happened in
# 2018 major, meaning that this gives us much less data to work with, since few policies were purchased after 2018

### But the average policy premium for these customers is higher than average. Again, with such a small number of customers, it is difficult to say definitively.

In [574]:
avg_policy_premium = policy_transactions_df.drop_duplicates(subset='customer_policy_id', keep='last')['transaction_amount'].mean()
policies_after_payout_with_premiums = pd.merge(policies_after_payout, policy_transactions_df, on='customer_policy_id').drop_duplicates(subset='customer_policy_id', keep='last')
avg_policy_after_payout_premium = policies_after_payout_with_premiums['transaction_amount'].mean()

In [594]:
avg_premiums = ['All Customers', 'Customers with Prior Payout']

trace1 = go.Bar(x=avg_premiums, y=[avg_policy_premium, avg_policy_after_payout_premium], name="Avg Premiums", marker=dict(color='green'))

data = [trace1]
layout = go.Layout(
    title='Avg Policy Premiums',
    yaxis=dict(
        title='GHS'
    )
)
pyo.iplot(go.Figure(data=data, layout=layout))

# Customers with a prior payout have higher premiums than average. However, the number of such customers is so small
# it may not be accurate

## Thank you for reading through this and for the opportunity to attempt this challenge. It was thoroughly enjoyable. I've left some additional thoughts below:

## Questions I would explore with more time:
### 1. What effect does type of crop have on retention rates? Are some crops more precious and therefore more likely to be insured?
### 2. How long does the effect of a payout last? Does a paid out customer's likelihood to renew diminish with time after a payout? Or is it static?
### 3. What effects do payout requests and payouts due have on retention? Is it a drought that makes a farmer purchase a policy, or is it the experience of receiving cash?
### 4. What are the characteristics of repeat customers - meaning ones who buy policies for consecutive seasons?
### 5. Some communities appear to have retention rates close to 100%. What is the reason behind this? What are the characteristics of these communities? Do they have large numbers of customers?
### 6. Do recent weather events (besides drought) influence retention? For example, does experiencing rain make customers less likely to take out a policy?
### 7. As a follow up to the previous question, in 2018, was it the experience of drought that made farmers take out policies, or was it the news that drought was expected?
### 8. I would try and get the lat/lon of communities that do not have them.

## Data/Data Model Concerns:
### 1. Not all policies have a corresponding policy_transaction, and there are sometimes duplicate policy_transactions for a single policy. policy_transactions should have a unique customer_policy_id.
### 2. I was unsure of the purpose of the community_payouts table. If payouts are made to individual farmers (rather than communities as a whole), maybe it makes more sense to have payouts linked to customers rather than communities. This could live in a table called customer_payouts, and could have a foreign key linking to the customers table. Customers have community ids, so we can easily do a group_by to recreate the community_payouts table.
### 3. Most customers have an unassigned gender. Ditto for having mobile money.
### 4. A large number of communities don't have a lat lon.
### 5. Almost all farmers have plots 13 acres in area. This seems suspicious to me.